In [2]:
import json
import os
from pathlib import Path

# Define the class names with subdivisions and their corresponding class IDs
class_names = [
    'a', 'b1', 'b2', 'b3', 'b4',  # Group 'b' has subdivisions b1, b2, b3, b4
    'c1', 'c2',                   # Group 'c' has subdivisions c1, c2
    'd1',                        # Group 'd' has d1
    'e1', 'e2', 'e3',             # Group 'e' has subdivisions e1, e2, e3
    'f1', 'f2',                   # Group 'f' has subdivisions f1, f2
    'g1', 'g2', 'g3',             # Group 'g' has subdivisions g1, g2, g3
    'h1', 'h2', 'h3',             # Group 'h' has subdivisions h1, h2, h3
    'i1'                         # Group 'i' has i1
]

# Function to calculate the bounding box from polygon points
def get_bounding_box(polygon_points):
    x_min = min(polygon_points, key=lambda point: point[0])[0]
    x_max = max(polygon_points, key=lambda point: point[0])[0]
    y_min = min(polygon_points, key=lambda point: point[1])[1]
    y_max = max(polygon_points, key=lambda point: point[1])[1]
    
    return x_min, y_min, x_max, y_max

# Function to convert a single JSON annotation to YOLOv5 txt format
def convert_json_to_yolo(json_file, image_width, image_height):
    with open(json_file, 'r') as f:
        data = json.load(f)

    output_txt = []
    
    for shape in data["shapes"]:
        label = shape["label"]
        points = shape["points"]
        
        if label in class_names:
            class_id = class_names.index(label)
            
            # Calculate bounding box from polygon points
            x_min, y_min, x_max, y_max = get_bounding_box(points)
            
            # Normalize the coordinates
            center_x = (x_min + x_max) / 2 / image_width
            center_y = (y_min + y_max) / 2 / image_height
            width = (x_max - x_min) / image_width
            height = (y_max - y_min) / image_height
            
            # Prepare the YOLO format entry
            output_txt.append(f"{class_id} {center_x} {center_y} {width} {height}")
    
    return output_txt

# Function to convert all JSON files in a given directory to YOLOv5 TXT
def convert_json_dir_to_yolo(json_dir, images_dir, output_dir):
    json_files = Path(json_dir).rglob("*.json")
    
    for json_file in json_files:
        # Get corresponding image name
        image_name = json_file.stem + ".jpg"  # Assuming images are in JPG format
        image_path = os.path.join(images_dir, image_name)
        
        if os.path.exists(image_path):
            # Get image dimensions (using PIL for example)
            from PIL import Image
            image = Image.open(image_path)
            image_width, image_height = image.size
            
            # Convert JSON to YOLOv5 format
            output_txt = convert_json_to_yolo(json_file, image_width, image_height)
            
            # Write to TXT file
            output_txt_file = os.path.join(output_dir, json_file.stem + ".txt")
            with open(output_txt_file, 'w') as out_f:
                out_f.write("\n".join(output_txt))

# Usage
json_dir = "C:\\Users\\Lenovo\\Documents\\yolo\\Labels"  
images_dir = "C:\\Users\\Lenovo\\Documents\\yolo\\dataset\\images"    # Directory with the corresponding images
output_dir = "C:\\Users\\Lenovo\\Documents\\yolo\\dataset\\labels"    # Directory to save the output .txt files

# Make sure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Convert all JSON files to YOLOv5 txt format
convert_json_dir_to_yolo(json_dir, images_dir, output_dir)
